In [ ]:
from PIL import Image 
import os 

In [ ]:
def append_images(images, direction='horizontal',
                  bg_color=(255,255,255), aligment='center'):
    """
    Appends images in horizontal/vertical direction.
    
    https://stackoverflow.com/questions/30227466/combine-several-images-horizontally-with-python/46623632#46623632
    
    Args:
        images: List of PIL images
        direction: direction of concatenation, 'horizontal' or 'vertical'
        bg_color: Background color (default: white)
        aligment: alignment mode if images need padding;
           'left', 'right', 'top', 'bottom', or 'center'

    Returns:
        Concatenated image as a new PIL image object.
    """
    widths, heights = zip(*(i.size for i in images))

    if direction=='horizontal':
        new_width = sum(widths)
        new_height = max(heights)
    else:
        new_width = max(widths)
        new_height = sum(heights)

    new_im = Image.new('RGB', (new_width, new_height), color=bg_color)


    offset = 0
    for im in images:
        if direction=='horizontal':
            y = 0
            if aligment == 'center':
                y = int((new_height - im.size[1])/2)
            elif aligment == 'bottom':
                y = new_height - im.size[1]
            new_im.paste(im, (offset, y))
            offset += im.size[0]
        else:
            x = 0
            if aligment == 'center':
                x = int((new_width - im.size[0])/2)
            elif aligment == 'right':
                x = new_width - im.size[0]
            new_im.paste(im, (x, offset))
            offset += im.size[1]

    return new_im

## Charts

In [ ]:
base_path = '/home/malcolm/petfinder/data/result_images/'

In [ ]:
latest_dir = os.listdir(base_path)
latest_dir.sort()
latest_dir = latest_dir[-1]
base_path = base_path + latest_dir + '/'

In [ ]:
images = os.listdir(base_path)
imgs = {img_name :Image.open(base_path + img_name) for img_name in images}
images

In [ ]:
img1 = append_images([imgs['Added Last Week Dogs.png'], imgs['Still There Dogs.png']]
                     , direction='horizontal')
img2 = append_images([imgs['N Dogs in Shelters.png'], imgs['N Days per Dog.png']], direction='horizontal')
img3 = append_images([img1, img2], direction='vertical')

In [ ]:
img3

## Individual Dogs

In [ ]:
import sqlite3 
import numpy as np
import pandas as pd
import datetime

In [ ]:
dogs_pic_folder = '/home/malcolm/sym_data_storage/Petfinder/Dogs/2021-12/'
image_save_folder = '/home/malcolm/petfinder/data/result_images/2021-12-31/'


In [ ]:
# 4 pictures of dogs to send 
total_dog_pics = os.listdir(dogs_pic_folder)
pics_to_send = np.random.choice([dogs_pic_folder + x for x in total_dog_pics], size = 4)

# Charts 
charts = os.listdir(image_save_folder)
image_paths_to_send = [image_save_folder + x for x in charts]
image_paths_to_send.extend(pics_to_send.tolist())

In [ ]:
len(image_paths_to_send)

In [ ]:
con = sqlite3.connect('/home/malcolm/petfinder/data/petfinder.db')
cursor = con.cursor()

In [ ]:
img_status = pd.read_sql(""" 
select * 
from Dog_Image_Status
limit 10 
""", con)
img_status.head()

In [ ]:
cursor.execute("""
select date_saved, count(*) as cnt
, count(distinct id) as unique_id
, count(distinct name) as unique_names
from Dog_Image_Status
where date_saved > '2021-12-15'
group by date_saved 
order by 1 desc
""").fetchall()

In [ ]:
today = datetime.datetime.now() - datetime.timedelta(days=1)
today_str = str(datetime.datetime.now().date())
two_weeks = today - datetime.timedelta(days=7)
two_weeks_str = str(two_weeks.date())

In [ ]:
cursor.execute(f"""
select count(distinct id) from dog_10475_30mi
where date_saved between '{two_weeks_str}' and '{today_str}'
""").fetchall()

In [ ]:
data_raw = pd.read_sql(f"""
select dogs.*, orgs.name as org_name 
from dog_10475_30mi dogs
left join Organizations orgs
on dogs.organization_id = orgs.id
where date_saved between '{two_weeks_str}' and '{today_str}'
""" , con)
print("Shape: ", data_raw.shape)
print(data_raw.columns)

In [ ]:
pd.set_option('max_columns', 1000)
pd.set_option('max_colwidth', 1000)

In [ ]:
key_cols = ['size', 'gender', 'tags', 'name', 'description', 'breeds.primary', 'contact.address.city'
           , 'org_name', 'date_saved']

In [ ]:
img_pics_df = pd.read_sql(f"""
select max(dogs.name) as dog_name
, max(`breeds.primary`) as breed, max(gender) as gender, max(size) as size
, max(`colors.primary`) as color
, max(orgs.name) as org_name, max(`contact.address.city`) as city, min(img.date_saved) as img_date_saved
, max(description) as description--, max(tags) tags
, max(Image_Status) img_loc, dogs.id
from dog_10475_30mi dogs
left join Organizations orgs
on dogs.organization_id = orgs.id
left join Dog_Image_Status img
on dogs.id = img.id
where img.date_saved between '{two_weeks_str}' and '{today_str}'
and img.Image_Status is not null 
and img.Image_Status != ''
group by dogs.id
order by dogs.id 
""" , con)
print("Shape: ", img_pics_df.shape)
print(img_pics_df.columns)
img_pics_df.head()

In [ ]:
img_pics_df.head()

In [ ]:
selected_pics = img_pics_df.sample(n=4, random_state=123)
pic_locs = selected_pics['img_loc'].tolist()
selected_pics = selected_pics.drop(['img_loc'], axis=1)
selected_pics

In [ ]:
dog_imgs = [Image.open(x) for x in pic_locs]
dog_img1 = append_images([dog_imgs[0], dog_imgs[1]]
                        , direction='horizontal')
dog_img2 = append_images([dog_imgs[2], dog_imgs[3]]
                        , direction='horizontal')
dog_img3 =  append_images([dog_img1, dog_img2], direction='vertical')
dog_img3

In [ ]:
dog_img3.save(image_save_folder+ '4_dogs.jpeg')

## Plotly to PIL

In [ ]:
import plotly.graph_objects as go

# Create figure
fig = go.Figure()

# Add trace
fig.add_trace(
    go.Scatter(x=[0, 0.5, 1, 2, 2.2], y=[1.23, 2.5, 0.42, 3, 1])
)

# Add images
fig.add_layout_image(
        dict(
            source="https://images.plot.ly/language-icons/api-home/python-logo.png",
            xref="x",
            yref="y",
            x=0,
            y=3,
            sizex=2,
            sizey=2,
            sizing="stretch",
            opacity=0.5,
            layer="below")
)

# Set templates
fig.update_layout(template="plotly_white")

fig.show()

In [ ]:
pil_img = fig.to_image()

In [ ]:
import io

In [ ]:
Image.open(io.BytesIO(pil_img))

## Test Changes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys 
import os
os.chdir('/home/malcolm')
sys.path.append('/home/malcolm/EmailSender/')
sys.path.append('/home/malcolm/petfinder/')
from EmailSender.EmailSender import EmailSender as ES
from pf_scripts.petfinder_eda import PetFinderEda
import datetime

today = datetime.datetime.now().date()


In [8]:
try: 

#     pf_eda = PetFinderEda()
#     pf_eda.execute()
    
    html_dict = {k:v.to_html() for k, v in pf_eda.output_dfs.items() }
    body_html = f"""
    This email contains a report of the <b>dogs</b> listed on Petfinder.com from {pf_eda.two_weeks_str} to {pf_eda.today_str} 
    for all locations 30 miles from 10475. 

    It contains summary information about the Number of dogs in shelters as of {pf_eda.today_str}, top organizations
    , dogs who have been in shelters the longest. 
    <br>
    <br>
    <b>Adoptions</b>
    {html_dict['New Old Existing']}
    <br>
    <br>
    <b>Time in Shelter </b>
    {html_dict['Long Ad Posted DF']}
    <br>
    <br>
    <b>Shelters with Most Dogs </b> 
    {html_dict['Top Orgs']}
    <br>
    <br>
    <b>Summary of Dogs in Shelter </b>
    {html_dict['All Summaries']}
    <br><br>
    
    <b>Dogs in Image</b> 
    {html_dict['Dogs in Imgs']}

    """
    
    message_params = {}
    message_params['Subject'] = f"Petfinder EDA {str(today)}"
    message_params['Body'] = body_html
    message_params['Image_paths'] = pf_eda.image_paths_to_send    
    
    email_sender = ES(**message_params)
    email_sender.execute()
    print("Success")
except Exception as e:
    error_email = {}
    error_email['Subject'] = f"Error in Petfinder for {str(today)}"
    error_email['Body'] = "Error in Petfinder: Error message <br><br>" + str(e)
    
    email_sender = ES(**error_email)
    email_sender.execute()
    print("Failed")

Success
